# Requierments


In [1]:
!pip install findspark
!pip install confluent-kafka
# Downloaded from https://mvnrepository.com/artifact/org.apache.spark/spark-streaming-kafka-0-8-assembly_2.11
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar

--2020-04-24 17:59:28--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.112.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.112.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13385346 (13M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.1’

spark-streaming-kaf 100%[===================>]  12.76M  4.96MB/s    in 2.6s    

2020-04-24 17:59:31 (4.96 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.1’ saved [13385346/13385346]



In [2]:
import os
import findspark
findspark.init('/usr/local/spark/spark-2.4.0-bin-hadoop2.7')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell'

---

In [3]:
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizerModel, IDFModel, StandardScalerModel, Tokenizer
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.streaming.kafka import KafkaUtils

from urllib.parse import unquote

APP_NAME = "BigData"
conf = pyspark.SparkConf().setAll([ ('spark.app.name', APP_NAME),
                                    ('spark.executor.memory', '8g'),
                                    ('spark.cores.max', '2'),
                                    ('spark.driver.memory','8g'),
                                   ('spark.master', 'local[2]')])
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)
sc

<SparkContext master=local[2] appName=BigData>

In [ ]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = ''
    for i in range(0,len(payload)-n + 1):
        ngrams += payload[i:i+n]+ ' '
    return ngrams[:-1]

ngrams = udf(to_ngram, StringType())
tokenizer = Tokenizer.load('../models/Tokenizer')
vectorizer = CountVectorizerModel.load('../models/Vectorizer')
idf_model = IDFModel.load('../models/idf')
scalerModel = StandardScalerModel.load('../models/scalerModel')
model = LogisticRegressionModel.load('../models/Logistic_Regression_Model')


topic = "test"
brokers = "localhost:9092"
ssc = StreamingContext(sc, batchDuration= 3)
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})


print("ready to run")

ready to run


In [ ]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = []
    for i in range(0,len(payload)-n+1):
        ngrams.append(payload[i:i+n])
    return ngrams


def get_prediction(queries):

    try:
        queries = queries.map(lambda w: Row(payload=w))
        queries = sqlc.createDataFrame(queries)
        queries = queries.withColumn('ngrams', ngrams(queries['payload']))
        queries = tokenizer.transform(queries)
        queries = vectorizer.transform(queries)
        queries = idf_model.transform(queries)
        queries = scalerModel.transform(queries)
        preds = model.transform(queries)
        preds.select('payload','prediction').show(truncate=False)
    except : 
        print('No data')
    

In [ ]:
queries = kvs.map(lambda x: unquote(x[1]))
# ngrams = queries.map(lambda x: to_ngram(x))
queries.foreachRDD(get_prediction)


ssc.start()
ssc.awaitTermination()  

No data
No data
No data
No data
+--------------------------------------+----------+
|payload                               |prediction|
+--------------------------------------+----------+
|/tagarela 1/                          |0.0       |
|/realplayer10/                        |0.0       |
|/6404573/                             |0.0       |
|/000023953/                           |0.0       |
|/_mocks/                              |0.0       |
|/javascript/oaerrordetailpage.svn-base|0.0       |
|/041440/                              |0.0       |
|/mgnews/                              |0.0       |
|/123004/                              |0.0       |
|/nettoys/                             |0.0       |
|/flag_japan/                          |0.0       |
|/666865/                              |0.0       |
|/136421/                              |0.0       |
|/javascript/todo.trace                |0.0       |
|/directory_com-coo/                   |0.0       |
|/seagate pro firewall  an crack